# Dresden und KI

Es soll ein RAG System für das Befragen der "Künstliche Intelligenz"-Seite von dresden.de erstellt werden.

In [8]:
base_url = "https://www.dresden.de/de/wirtschaft/tomorrow-s-home/kuenstliche-intelligenz.php"
content_class = "contentsection-container "

In [9]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=(base_url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=(content_class)
        )
    ),
)
docs = loader.load()

In [10]:
len(docs)

1

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [13]:
len(splits)

21

In [15]:
from utils import load_openai_key
from langchain_openai import ChatOpenAI

load_openai_key()
llm = ChatOpenAI(model="gpt-4o")

In [16]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. If you used a specific piece of context, clearly say so"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [24]:
rag_chain.invoke({"input": "Was sind aktuelle KI Themen in Dresden?"})

{'input': 'Was sind aktuelle KI Themen in Dresden?',
 'context': [Document(page_content='Ob Suchmaschinen, Smart Citys oder autonomes Fahren – Künstliche Intelligenz (KI) findet auch im Alltag eine immer breitere Anwendung. In der KI-Metropole Dresden werden dafür die Voraussetzungen geschaffen. Die sächsische Landeshauptstadt ist einer der führenden Hochtechnologiestandorte Europas und an der Weiterentwicklung von kritischer KI-Hardware, Halbleitern und KI-Software beteiligt. Die hohe Dichte von exzellenten Unternehmen, Start-ups und Forschungseinrichtungen, die sich am Standort angesiedelt haben, macht Dresden zu einem einmaligen Cluster der Künstlichen Intelligenz.\n\n\n\n\n\n\n\n\n\n\n\n\nDas CeTI der TU Dresden: Künstliche Intelligenz zum Anfassen\n\n\n\n\n\n\n\n\n\n\n\n© CeTI Dresden, intoVR', metadata={'source': 'https://www.dresden.de/de/wirtschaft/tomorrow-s-home/kuenstliche-intelligenz.php'}),
  Document(page_content='Kollaboratives Arbeiten von Menschen und Robotern könnte i

In [25]:
rag_chain.invoke({"input": "Wie viele Prozessschritte sind für die Wafer von Bosch notwendig?"})

{'input': 'Wie viele Prozessschritte sind für die Wafer von Bosch notwendig?',
 'context': [Document(page_content='Rund eine Milliarde Euro hat Bosch in seine Dresdner „Waferfab“ investiert – mehr als jemals zuvor in der traditionsreichen Geschichte des Unternehmens. Das Ergebnis: eine der modernsten Chip-Fabriken der Welt. Produziert werden 300-Milimeter-Wafer aus Silizium für das Internet der Dinge und die Mobilität der Zukunft. Dabei setzt Bosch auf eine vollautomatisierte Fertigung mit AIoT-Technologie, also einer Kombination aus Künstlicher Intelligenz und dem Internet der Dinge. Damit schafft Bosch die Grundlage für eine datengesteuerte, kontinuierliche Verbesserung in der Produktion. So können zum Beispiel kleinste Fehler (Signaturen) bei der Wafer-Fertigung durch einen KI-Algorithmus erkannt, die Fehlerursachen analysiert und Prozessabweichungen unmittelbar korrigiert werden. Der Fertigungsprozess und die Qualität der Halbleiter werden auf diese Weise stetig verbessert, der Gra